# Notebook for developing python and testing

In [2]:
import requests as r
import json
import pandas as pd
import pyodbc




In [5]:
url_w = "https://api.weatherapi.com/v1/current.json?q=Perth&key=96792ecea70641218aa32831252402" #key should be piped but just making something quickly

conn = ""


response = r.get(url=url_w)
data_dict = json.loads(response.content)
data = pd.json_normalize(data_dict)

In [59]:
import datetime as dt


today = dt.date.today() - dt.timedelta(days=1)

In [60]:
today

datetime.date(2025, 2, 24)

In [ ]:
dt.(today, 1)

NotImplemented

In [61]:
import datetime as dt

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

theday = dt.date.today() - dt.timedelta(days=1)
day =  theday.day
month = months[theday.month-1]

year = theday.year
if day < 10:
    day = f"0{str(day)}"

date_ref = f"{year}-{month}-{day}"

In [62]:
date_ref
url2 = f"https://api.weatherapi.com/v1/history.json?q=Perth&dt={date_ref}&end_dt={date_ref}&hour=8&key=96792ecea70641218aa32831252402"


In [20]:
url2

'https://api.weatherapi.com/v1/history.json?q=Perth&dt=2025-Feb-24&end_dt=2025-Feb-24&hour=8&key=96792ecea70641218aa32831252402'

In [21]:
response = r.get(url=url2)
data_dict = json.loads(response.content)
data = pd.json_normalize(data_dict['forecast']['forecastday'])


In [22]:
data

,date,date_epoch,hour,day.maxtemp_c,day.maxtemp_f,day.mintemp_c,day.mintemp_f,day.avgtemp_c,day.avgtemp_f,day.maxwind_mph,...,day.condition.text,day.condition.icon,day.condition.code,day.uv,astro.sunrise,astro.sunset,astro.moonrise,astro.moonset,astro.moon_phase,astro.moon_illumination
0,2025-02-24,1740355200,"[{'time_epoch': 1740355200, 'time': '2025-02-2...",24.8,76.6,19.6,67.4,22.0,71.7,12.8,...,Sunny,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,9.0,06:02 AM,06:57 PM,01:29 AM,04:34 PM,Waning Crescent,22


In [23]:
data.columns = [
'date',
'date_epoch',
'hour',
'maxtemp_c',
'maxtemp_f',
'mintemp_c',
'mintemp_f',
'avgtemp_c',
'avgtemp_f',
'maxwind_mph',
'maxwind_kph',
'totalprecip_mm',
'totalprecip_in',
'totalsnow_cm',
'avgvis_km',
'avgvis_miles',
'avghumidity',
'daily_will_it_rain',
'daily_chance_of_rain',
'daily_will_it_snow',
'daily_chance_of_snow',
'condition_text',
'condition_icon',
'condition_code',
'uv',
'sunrise',
'sunset',
'moonrise',
'moonset',
'moon_phase',
'moon_illumination',
]

In [26]:
data.iloc[0, 2] = 8

In [27]:
data

,date,date_epoch,hour,maxtemp_c,maxtemp_f,mintemp_c,mintemp_f,avgtemp_c,avgtemp_f,maxwind_mph,...,condition_text,condition_icon,condition_code,uv,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,2025-02-24,1740355200,8,24.8,76.6,19.6,67.4,22.0,71.7,12.8,...,Sunny,//cdn.weatherapi.com/weather/64x64/day/113.png,1000,9.0,06:02 AM,06:57 PM,01:29 AM,04:34 PM,Waning Crescent,22


<!-- Make sure the data is in the correct format etc. -->

In [39]:
connection = "Server=joshwork\\MSSQLSERVER02;Database=db1;user=sa;password=password1;Trusted_Connection=True;TrustServerCertificate=True"


connection_string = "mssql+pyodbc://@joshwork\\MSSQLSERVER02/db1?driver=ODBC+Driver+17+for+SQL+Server&Trusted_Connection=yes&TrustServerCertificate=yes"



In [67]:
theDate = dt.date(year, theday.month, theday.day)

In [63]:
theday

datetime.date(2025, 2, 24)

In [68]:
data.iloc[0,0] = theDate

Mostly finished - just need to work out how to get the datatype to map correctly to SQL Date, if this fails may need to do SQL command generation as I have done previously

In [70]:
data.to_sql( 
    name="WeatherDataModelsTest",
    con=connection_string, 
    if_exists="append",
    index=False,
    # index_label="Id"
    )

1